<a href="https://colab.research.google.com/github/PranjalSahu/OAI_analysis_2/blob/pranjal/DaskComputationCoiled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install packages

# Remember to Restart runtime after installation

!pip install pandas==1.4.2 
!pip install itk==5.3rc4
!pip install vtk
!pip install itkwidgets
!pip install icon-registration==0.3.4
#!pip install "dask[complete]"
#!pip install coiled dask distributed
!pip install tornado==6.1
!pip install coiled
!pip install torch
!pip install jupyter
!pip install girder_client
!pip install git+https://github.com/uncbiag/mermaid.git
!pip install git+https://github.com/uncbiag/easyreg.git
!pip install git+https://github.com/PranjalSahu/OAI_analysis_2.git#egg=oai_package

In [ ]:
# All Imports

import numpy as np
import itk
import vtk
import itkwidgets
import icon_registration
import icon_registration.itk_wrapper as itk_wrapper
import icon_registration.pretrained_models as pretrained_models
from oai_analysis_2 import mesh_processing as mp

import os
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import boto3
import coiled
import dask
from dask import delayed, compute, visualize
from dask.distributed import Client, progress, LocalCluster

In [ ]:
# For creating GPU instances
# Start with a docker container and then install the rest of packages
# coiled.create_software_environment(
#     name="gpu-test",
#     container="gpuci/miniconda-cuda:11.2-runtime-ubuntu20.04",
#     pip="coiled_requirements.txt",
# )

# coiled.create_software_environment(
#     name="gpu-test2",
#     container="pranjalsahu/pranjal-sahu-gpu-test1",
# )

# coiled.create_software_environment(
#     name="gpu-test1",
#     container="gpuci/miniconda-cuda:10.2-runtime-ubuntu18.04",
#     conda={
#         "channels": ["conda-forge", "rapidsai", "defaults"],
#         "dependencies": ["dask", "dask-cuda", "cupy", "cudatoolkit=10.2"],
#     },
#     pip="coiled_requirements.txt"
# )


Creating new software environment
Creating container-only software environment
Successfully saved software environment build


In [ ]:
# Create the environment

if False:
  coiled.create_software_environment(
    name="oai6",
    pip="coiled_requirements.txt",
  )
  # coiled.create_software_environment(
  #   name="oai4",
  #   container="pranjalsahu/pranjal-sahu-oai4:cf409eeb-435c-4976-afd4-6a5118d6b683",
  # )

In [ ]:
# To create a software environment using docker with open gl installed
# coiled.create_software_environment(
#     name="oai6",
#     container="pranjalsahu/pranjal-sahu-oai6:latest",
# )

Creating new software environment
Creating container-only software environment
Successfully saved software environment build


In [ ]:
# Create Dask Client. This will also spawn dask worker and scheduler

use_coiled = True

if use_coiled:
  name = 'oai-processing'
  cluster = coiled.Cluster(n_workers=2,
                         worker_cpu=8,
                         worker_memory='31G',
                         name=name,
                         worker_gpu=1,
                         worker_class="dask_cuda.CUDAWorker",
                         software='pranjal-sahu/gpu-test2')
  client = dask.distributed.Client(cluster, 
                                   serializers=['pickle', 'dask'],
                                   deserializers=['pickle', 'dask'])
  client
else:
  cluster = LocalCluster(processes=False)
  client = Client(cluster)
  client

Output()

Found software environment build
Created fw rule: inbound [8786-8787] [0.0.0.0/0] []
Created FW rules: coiled-dask-pranjal09-152057-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-pranjal09-152057-firewall -> coiled-dask-pranjal09-152057-firewall]
Created FW rules: coiled-dask-pranjal09-152057-cluster-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-pranjal09-152057-cluster-firewall -> coiled-dask-pranjal09-152057-cluster-firewall]
Created scheduler VM: coiled-dask-pranjal09-152057-scheduler (type: t3a.medium, ip: ['54.196.128.174'])


Mismatched versions found

+-------------+----------------+----------------+----------------+
| Package     | client         | scheduler      | workers        |
+-------------+----------------+----------------+----------------+
| dask        | 2022.02.0      | 2022.03.0      | 2022.03.0      |
| distributed | 2022.02.0      | 2022.3.0       | 2022.3.0       |
| lz4         | None           | 4.0.0          | 4.0.0          |
| pandas      | 1.3.5          | 1.4.2          | 1.4.2          |
| python      | 3.7.13.final.0 | 3.9.12.final.0 | 3.9.12.final.0 |
+-------------+----------------+----------------+----------------+


In [ ]:
cluster.close()

Cluster deleted successfully.

In [ ]:
@delayed
def test_gpu():
    import numpy as np
    import cupy as cp

    x = cp.arange(6).reshape(2, 3).astype("f")
    return cp.asnumpy(x.sum())


#client = Client(cluster)
#f = client.submit(test_gpu)
#f.result()

In [ ]:
r1 = [test_gpu()]
compute(r1)

distributed.protocol.pickle - INFO - Failed to deserialize b'\x80\x05\x95\xfa\x03\x00\x00\x00\x00\x00\x00\x8c\x16tblib.pickling_support\x94\x8c\x12unpickle_exception\x94\x93\x94(\x8c\x08builtins\x94\x8c\tTypeError\x94\x93\x94\x8c\'an integer is required (got type bytes)\x94\x85\x94Nh\x00\x8c\x12unpickle_traceback\x94\x93\x94\x8c\x05tblib\x94\x8c\x05Frame\x94\x93\x94)\x81\x94}\x94(\x8c\x08f_locals\x94}\x94\x8c\tf_globals\x94}\x94(\x8c\x08__name__\x94\x8c\x12distributed.worker\x94\x8c\x08__file__\x94\x8cH/opt/conda/envs/coiled/lib/python3.9/site-packages/distributed/worker.py\x94u\x8c\x06f_code\x94h\n\x8c\x04Code\x94\x93\x94)\x81\x94}\x94(\x8c\x0bco_filename\x94h\x16\x8c\x07co_name\x94\x8c\x17_maybe_deserialize_task\x94\x8c\x0bco_argcount\x94K\x00\x8c\x11co_kwonlyargcount\x94K\x00\x8c\x0bco_varnames\x94)\x8c\nco_nlocals\x94K\x00\x8c\x0cco_stacksize\x94K\x00\x8c\x08co_flags\x94K@\x8c\x0eco_firstlineno\x94K\x00ub\x8c\x08f_lineno\x94M\x14\rubM\t\rh\n\x8c\tTraceback\x94\x93\x94)\x81\x94}\x94

KeyboardInterrupt: ignored

In [ ]:
# Download the relevant files such as input image, atlas image, segmented TC and FC maps

!girder-client --api-url https://data.kitware.com/api/v1 localsync 621a5d784acac99f426c143b OAIData

Local metadata does not exists. Falling back to download.
[####################################]  74.74M/74.74M  100%  atlas_image.nii.gz          
[####################################]  76.70M/76.70M  100%  FC_probmap.nii.gz          
[####################################]  27.96M/27.96M  100%  image_preprocessed.nii.gz          
[####################################]  76.66M/76.66M  100%  TC_probmap.nii.gz          


In [ ]:
# All Function Definitions with Dask Delayed Decorator to perform parallel Computing

#@delayed(nout=2)
def read_images(image_preprocessed, atlas_image):
    image_A = itk.imread(image_preprocessed, itk.D)
    image_B = itk.imread(atlas_image, itk.D)

    fc_prob_file = './OAIData/FC_probmap.nii.gz'
    FC_prob = itk.imread(fc_prob_file)

    tc_prob_file = './OAIData/TC_probmap.nii.gz'
    TC_prob = itk.imread(tc_prob_file)

    return image_A, image_B, FC_prob, TC_prob

@delayed(nout=3)
def register_images_delayed():
    import boto3
    import itk
    import icon_registration
    import icon_registration.itk_wrapper as itk_wrapper
    import icon_registration.pretrained_models as pretrained_models
    from os.path import exists

    if 1:
        image_A = 'image_preprocessed.nii.gz'
        image_B = 'atlas_image.nii.gz'
        if (exists(image_A) and exists(image_B)) == False:
          s3          = boto3.resource("s3")
          bucket_name = 'oaisample1'
          bucket      = s3.Bucket(bucket_name)

          s3.Bucket(bucket_name).download_file(image_A, image_A)
          s3.Bucket(bucket_name).download_file(image_B, image_B)
    else:
        image_A = './OAIData/image_preprocessed.nii.gz'
        image_B = './OAIData/atlas_image.nii.gz'

    image_A = itk.imread(image_A, itk.D)
    image_B = itk.imread(image_B, itk.D)

    model = pretrained_models.OAI_knees_gradICON_model()
    model.to('cpu')

    # Register the images
    phi_AB, phi_BA = itk_wrapper.register_pair(model, image_A, image_B)
    return itk.dict_from_transform(phi_AB), itk.dict_from_image(image_A), itk.dict_from_image(image_B)

@delayed(nout=1)
def deform_probmap_FC_delayed(phi_AB, image_A, image_B):
    import itk
    import boto3

    if 1:
        s3          = boto3.resource("s3")
        bucket_name = 'oaisample1'
        bucket      = s3.Bucket(bucket_name)
        fc_prob_file = 'FC_probmap.nii.gz'
        s3.Bucket(bucket_name).download_file(fc_prob_file, fc_prob_file)

    phi_AB1  = itk.transform_from_dict(phi_AB)
    
    def set_parameters(phi_AB, phi_AB1):
        for i in range(len(phi_AB)-1):
            transform1 = phi_AB1.GetNthTransform(i)

            fp = phi_AB[i+1]['fixedParameters']
            o1 = transform1.GetFixedParameters()
            o1.SetSize(fp.shape[0])
            for j, v in enumerate(fp):
                o1.SetElement(j, v)
            transform1.SetFixedParameters(o1)

            p = phi_AB[i+1]['parameters']
            o2 = transform1.GetParameters()
            o2.SetSize(p.shape[0])
            for j, v in enumerate(p):
                o2.SetElement(j, v)
            transform1.SetParameters(o2)

    set_parameters(phi_AB, phi_AB1)
    image_A = itk.image_from_dict(image_A)
    image_B = itk.image_from_dict(image_B)

    interpolator = itk.LinearInterpolateImageFunction.New(image_A)
    
    prob_file = 'FC_probmap.nii.gz'
    prob = itk.imread(prob_file, itk.D)

    warped_image = itk.resample_image_filter(prob, 
       transform=phi_AB1, 
       interpolator=interpolator,
       size=itk.size(image_B),
       output_spacing=itk.spacing(image_B),
       output_direction=image_B.GetDirection(),
       output_origin=image_B.GetOrigin()
    )

    output_dict = itk.dict_from_image(warped_image)
    return output_dict
  
@delayed
def deform_probmap_TC_delayed(phi_AB, image_A, image_B):
    import itk
    import boto3

    if 1:
        s3          = boto3.resource("s3")
        bucket_name = 'oaisample1'
        bucket      = s3.Bucket(bucket_name)
        fc_prob_file = 'TC_probmap.nii.gz'
        s3.Bucket(bucket_name).download_file(fc_prob_file, fc_prob_file)

    phi_AB1  = itk.transform_from_dict(phi_AB)
    
    def set_parameters(phi_AB, phi_AB1):
        for i in range(len(phi_AB)-1):
            transform1 = phi_AB1.GetNthTransform(i)

            fp = phi_AB[i+1]['fixedParameters']
            o1 = transform1.GetFixedParameters()
            o1.SetSize(fp.shape[0])
            for j, v in enumerate(fp):
                o1.SetElement(j, v)
            transform1.SetFixedParameters(o1)

            p = phi_AB[i+1]['parameters']
            o2 = transform1.GetParameters()
            o2.SetSize(p.shape[0])
            for j, v in enumerate(p):
                o2.SetElement(j, v)
            transform1.SetParameters(o2)

    set_parameters(phi_AB, phi_AB1)
    image_A = itk.image_from_dict(image_A)
    image_B = itk.image_from_dict(image_B)

    interpolator = itk.LinearInterpolateImageFunction.New(image_A)
    
    prob_file = 'TC_probmap.nii.gz'
    prob = itk.imread(prob_file, itk.D)

    warped_image = itk.resample_image_filter(prob, 
       transform=phi_AB1, 
       interpolator=interpolator,
       size=itk.size(image_B),
       output_spacing=itk.spacing(image_B),
       output_direction=image_B.GetDirection(),
       output_origin=image_B.GetOrigin()
    )

    output_dict = itk.dict_from_image(warped_image)
    return output_dict
  

@delayed
def deform_probmap_TC(phi_AB, image_A, image_B, TC_prob):
    #import itkConfig
    #itkConfig.LazyLoading = False
    #import itk
    #tc_prob_file = './OAIData/TC_probmap.nii.gz'
    #TC_prob = itk.imread(tc_prob_file)

    interpolator = itk.LinearInterpolateImageFunction.New(image_A)
    warped_image_TC = itk.resample_image_filter(TC_prob, 
        transform=phi_AB, 
        interpolator=interpolator,
        size=itk.size(image_B),
        output_spacing=itk.spacing(image_B),
        output_direction=image_B.GetDirection(),
        output_origin=image_B.GetOrigin()
    )

    return warped_image_TC

# Convert from VTK Mesh to ITK Mesh to make it serializable
def get_itk_mesh(vtk_mesh):
    #import itkConfig
    #itkConfig.LazyLoading = False
    #import itk

    Dimension = 3
    PixelType = itk.D
    
    MeshType = itk.Mesh[PixelType, Dimension]
    itk_mesh = MeshType.New()
    
    # Get points array from VTK mesh
    points = vtk_mesh.GetPoints().GetData()
    points_numpy = np.array(points).flatten().astype('float32')
        
    polys = vtk_mesh.GetPolys().GetData()
    polys_numpy = np.array(polys).flatten()

    # Triangle Mesh
    vtk_cells_count = vtk_mesh.GetNumberOfPolys()
    polys_numpy = np.reshape(polys_numpy, [vtk_cells_count, Dimension+1])

    # Extracting only the points by removing first column that denotes the VTK cell type
    polys_numpy = polys_numpy[:, 1:]
    polys_numpy = polys_numpy.flatten().astype(np.uint64)

    # Get point data from VTK mesh to insert in ITK Mesh
    point_data_numpy = np.array(vtk_mesh.GetPointData().GetScalars())#.astype('float64')
    
    # Get cell data from VTK mesh to insert in ITK Mesh
    cell_data_numpy = np.array(vtk_mesh.GetCellData().GetScalars())#.astype('float64')
    
    itk_mesh.SetPoints(itk.vector_container_from_array(points_numpy))
    itk_mesh.SetCellsArray(itk.vector_container_from_array(polys_numpy), itk.CommonEnums.CellGeometry_TRIANGLE_CELL)
    itk_mesh.SetPointData(itk.vector_container_from_array(point_data_numpy))
    itk_mesh.SetCellData(itk.vector_container_from_array(cell_data_numpy))    
    return itk_mesh
    
@delayed
def get_thickness_FC(warped_image_FC):
    import itk
    from oai_analysis_2 import mesh_processing as mp

    warped_image_FC = itk.image_from_dict(warped_image_FC)
    distance_inner_FC, distance_outer_FC = mp.get_thickness_mesh(warped_image_FC, mesh_type='FC')
    distance_inner_FC = get_itk_mesh(distance_inner_FC)
    distance_inner_FC = itk.dict_from_mesh(distance_inner_FC)
    return distance_inner_FC

@delayed(nout=1)
def get_thickness_TC(warped_image_TC):
    import itk
    import vtk
    import numpy as np
    from oai_analysis_2 import mesh_processing as mp

    def get_itk_mesh(vtk_mesh):
        Dimension = 3
        PixelType = itk.D
        
        # Get points array from VTK mesh
        points = vtk_mesh.GetPoints().GetData()
        points_numpy = np.array(points).flatten()#.astype('float32')
            
        polys = vtk_mesh.GetPolys().GetData()
        polys_numpy = np.array(polys).flatten()

        # Triangle Mesh
        vtk_cells_count = vtk_mesh.GetNumberOfPolys()
        polys_numpy = np.reshape(polys_numpy, [vtk_cells_count, Dimension+1])

        # Extracting only the points by removing first column that denotes the VTK cell type
        polys_numpy = polys_numpy[:, 1:]
        polys_numpy = polys_numpy.flatten().astype(np.uint64)

        # Get point data from VTK mesh to insert in ITK Mesh
        point_data_numpy = np.array(vtk_mesh.GetPointData().GetScalars())#.astype('float64')
        
        # Get cell data from VTK mesh to insert in ITK Mesh
        cell_data_numpy = np.array(vtk_mesh.GetCellData().GetScalars())#.astype('float64')
        
        MeshType = itk.Mesh[PixelType, Dimension]
        itk_mesh = MeshType.New()
        
        itk_mesh.SetPoints(itk.vector_container_from_array(points_numpy))
        itk_mesh.SetCellsArray(itk.vector_container_from_array(polys_numpy), itk.CommonEnums.CellGeometry_TRIANGLE_CELL)
        itk_mesh.SetPointData(itk.vector_container_from_array(point_data_numpy))
        itk_mesh.SetCellData(itk.vector_container_from_array(cell_data_numpy))    
        return itk_mesh

    warped_image_TC = itk.image_from_dict(warped_image_TC)
    distance_inner_TC, distance_outer_TC = mp.get_thickness_mesh(warped_image_TC, mesh_type='TC')
    
    polys = distance_inner_TC.GetPolys().GetData()
    polys_numpy = np.array(polys).flatten()

    distance_inner_TC_itk = get_itk_mesh(distance_inner_TC)
    distance_inner_TC_itk_dict = itk.dict_from_mesh(distance_inner_TC_itk)

    cells1 = distance_inner_TC_itk_dict["cells"][:5]
    cells2 = distance_inner_TC_itk["cells"][:5]

    return [distance_inner_TC_itk_dict, distance_inner_TC_itk_dict["cells"], cells1, cells2, polys_numpy]
    #cells_array = distance_inner_TC_itk.GetCellsArray()
    #cells_array = itk.array_from_vector_container(cells_array)
    #return cells_array
    #return [distance_inner_TC.GetNumberOfPoints(), distance_inner_TC.GetNumberOfCells()]
    #distance_inner_TC = itk.dict_from_mesh(distance_inner_TC)
    #return distance_inner_TC
    #return [cells1, cells2]

#@delayed(nout=2)
def get_paths():
    import numpy as np
    import itk
    import vtk
    import itkwidgets
    import icon_registration
    import icon_registration.itk_wrapper as itk_wrapper
    import icon_registration.pretrained_models as pretrained_models
    from oai_analysis_2 import mesh_processing as mp

    image_preprocessed = "./OAIData/image_preprocessed.nii.gz"
    atlas_image = "./OAIData/atlas_image.nii.gz"

    return image_preprocessed, atlas_image

#@delayed
def only_register_images(image_A, image_B):
  #import itkConfig
  #itkConfig.LazyLoading = False
  #import itk
  import numpy as np
  import itk
  import vtk
  import itkwidgets
  import icon_registration
  import icon_registration.itk_wrapper as itk_wrapper
  import icon_registration.pretrained_models as pretrained_models
  from oai_analysis_2 import mesh_processing as mp

  phi_AB  = register_images(image_A=image_A, image_B=image_B)
  return phi_AB

@delayed
def import_all():
  import numpy as np
  #import itkConfig
  #itkConfig.LazyLoading = False
  import itk
  import vtk
  import itkwidgets
  import icon_registration
  import icon_registration.itk_wrapper as itk_wrapper
  import icon_registration.pretrained_models as pretrained_models
  from oai_analysis_2 import mesh_processing as mp

  return True

In [ ]:
# For checking if vtk import works properly
@delayed
def check_vtk():
  import vtk
  return vtk.__version__

@delayed
def check_skimage():
  import skimage
  return skimage.__version__

t1 = check_skimage()
output_result = compute(t1)
print(output_result)

('0.18.3',)


In [ ]:
# Main execution block

phi_AB, image_A, image_B = register_images_delayed()
#deformed_fc = deform_probmap_FC_delayed(phi_AB, image_A, image_B)
deformed_tc = deform_probmap_TC_delayed(phi_AB, image_A, image_B)

#thickness_fc = get_thickness_FC(deformed_fc)
thickness_tc = get_thickness_TC(deformed_tc)

result = [thickness_tc]
#result = #[deformed_fc]
output_result = compute(*result)
#print(output_result)

In [ ]:
print(output_result)

([{'meshType': {'dimension': 3, 'pointComponentType': 'float32', 'pointPixelComponentType': 'float64', 'pointPixelType': 'Scalar', 'pointPixelComponents': 1, 'cellComponentType': 'uint64', 'cellPixelComponentType': 'float64', 'cellPixelType': 'Scalar', 'cellPixelComponents': 1}, 'name': '', 'dimension': 3, 'numberOfPoints': 16587, 'points': array([ 55.538155,  91.491844,  64.33265 , ..., 104.95681 ,  93.99789 ,
        34.300396], dtype=float32), 'numberOfPointPixels': 16587, 'pointData': array([0.16583272, 0.        , 0.        , ..., 0.        , 0.10137601,
       0.        ]), 'numberOfCells': 32224, 'cells': array([    2,     3,     0, ..., 16584, 16586, 16585], dtype=uint64), 'numberOfCellPixels': 32224, 'cellData': array([0.05527757, 0.10370194, 0.05577836, ..., 0.09223103, 0.033792  ,
       0.03447352]), 'cellBufferSize': 161120}, array([    2,     3,     0, ..., 16584, 16586, 16585], dtype=uint64), array([2, 3, 0, 1, 2], dtype=uint64), array([2, 3, 0, 1, 2], dtype=uint64), arr

In [ ]:
print(output_result)

([{'meshType': {'dimension': 3, 'pointComponentType': 'float32', 'pointPixelComponentType': 'float64', 'pointPixelType': 'Scalar', 'pointPixelComponents': 1, 'cellComponentType': 'uint64', 'cellPixelComponentType': 'float64', 'cellPixelType': 'Scalar', 'cellPixelComponents': 1}, 'name': '', 'dimension': 3, 'numberOfPoints': 16587, 'points': array([ 55.538155,  91.491844,  64.33265 , ..., 104.95681 ,  93.99789 ,
        34.300396], dtype=float32), 'numberOfPointPixels': 16587, 'pointData': array([0.16583272, 0.        , 0.        , ..., 0.        , 0.10137601,
       0.        ]), 'numberOfCells': 32224, 'cells': array([    2,     3,     0, ..., 16584, 16586, 16585], dtype=uint64), 'numberOfCellPixels': 32224, 'cellData': array([6.93652245e-310, 6.93652258e-310, 0.00000000e+000, ...,
       9.22310260e-002, 3.37920043e-002, 3.44735188e-002]), 'cellBufferSize': 161120}, array([    2,     3,     0, ..., 16584, 16586, 16585], dtype=uint64), array([2, 3, 0, 1, 2], dtype=uint64), array([2, 3

In [ ]:
print(output_result)

([{'meshType': {'dimension': 3, 'pointComponentType': 'float32', 'pointPixelComponentType': 'float64', 'pointPixelType': 'Scalar', 'pointPixelComponents': 1, 'cellComponentType': 'uint64', 'cellPixelComponentType': 'float64', 'cellPixelType': 'Scalar', 'cellPixelComponents': 1}, 'name': '', 'dimension': 3, 'numberOfPoints': 16587, 'points': array([ 55.538155,  91.491844,  64.33265 , ..., 104.95681 ,  93.99789 ,
        34.300396], dtype=float32), 'numberOfPointPixels': 16587, 'pointData': array([0.16583272, 0.        , 0.        , ..., 0.        , 0.10137601,
       0.        ]), 'numberOfCells': 32224, 'cells': array([    2,     3,     0, ..., 16584, 16586, 16585], dtype=uint64), 'numberOfCellPixels': 32224, 'cellData': array([6.93652245e-310, 6.93652258e-310, 0.00000000e+000, ...,
       9.22310260e-002, 3.37920043e-002, 3.44735188e-002]), 'cellBufferSize': 161120}, array([    2,     3,     0, ..., 16584, 16586, 16585], dtype=uint64), array([2, 3, 0, 1, 2], dtype=uint64), array([2, 3

In [ ]:
print(output_result)

([{'meshType': {'dimension': 3, 'pointComponentType': 'float32', 'pointPixelComponentType': 'float64', 'pointPixelType': 'Scalar', 'pointPixelComponents': 1, 'cellComponentType': 'uint64', 'cellPixelComponentType': 'float64', 'cellPixelType': 'Scalar', 'cellPixelComponents': 1}, 'name': '', 'dimension': 3, 'numberOfPoints': 16587, 'points': array([ 55.538155,  91.491844,  64.33265 , ..., 104.95681 ,  93.99789 ,
        34.300396], dtype=float32), 'numberOfPointPixels': 16587, 'pointData': array([0.16583272, 0.        , 0.        , ..., 0.        , 0.10137601,
       0.        ]), 'numberOfCells': 32224, 'cells': array([140396306901776, 140396307045680,               0, ...,
                 16584,           16586,           16585], dtype=uint64), 'numberOfCellPixels': 32224, 'cellData': array([6.93616358e-310, 6.93649924e-310, 0.00000000e+000, ...,
       9.22310260e-002, 3.37920043e-002, 3.44735188e-002]), 'cellBufferSize': 161120}, array([140396306901776, 140396307045680,            

In [ ]:
print(output_result)

([{'meshType': {'dimension': 3, 'pointComponentType': 'float32', 'pointPixelComponentType': 'float64', 'pointPixelType': 'Scalar', 'pointPixelComponents': 1, 'cellComponentType': 'uint64', 'cellPixelComponentType': 'float64', 'cellPixelType': 'Scalar', 'cellPixelComponents': 1}, 'name': '', 'dimension': 3, 'numberOfPoints': 16587, 'points': array([ 55.538155,  91.491844,  64.33265 , ..., 104.95681 ,  93.99789 ,
        34.300396], dtype=float32), 'numberOfPointPixels': 16587, 'pointData': array([0.16583272, 0.        , 0.        , ..., 0.        , 0.10137601,
       0.        ]), 'numberOfCells': 32224, 'cells': array([    2,     3,     0, ..., 16584, 16586, 16585], dtype=uint64), 'numberOfCellPixels': 32224, 'cellData': array([6.93652251e-310, 6.93652245e-310, 0.00000000e+000, ...,
       9.22310260e-002, 3.37920043e-002, 3.44735188e-002]), 'cellBufferSize': 161120}, array([    2,     3,     0, ..., 16584, 16586, 16585], dtype=uint64), array([2, 3, 0, 1, 2], dtype=uint64), array([2, 3

In [ ]:
print(output_result)

([{'meshType': {'dimension': 3, 'pointComponentType': 'float32', 'pointPixelComponentType': 'float64', 'pointPixelType': 'Scalar', 'pointPixelComponents': 1, 'cellComponentType': 'uint64', 'cellPixelComponentType': 'float64', 'cellPixelType': 'Scalar', 'cellPixelComponents': 1}, 'name': '', 'dimension': 3, 'numberOfPoints': 16587, 'points': array([ 55.538155,  91.491844,  64.33265 , ..., 104.95681 ,  93.99789 ,
        34.300396], dtype=float32), 'numberOfPointPixels': 16587, 'pointData': array([0.16583272, 0.        , 0.        , ..., 0.        , 0.10137601,
       0.        ]), 'numberOfCells': 32224, 'cells': array([140396173240640, 140396172711552,               0, ...,
                 16584,           16586,           16585], dtype=uint64), 'numberOfCellPixels': 32224, 'cellData': array([6.93649257e-310, 6.93649261e-310, 0.00000000e+000, ...,
       9.22310260e-002, 3.37920043e-002, 3.44735188e-002]), 'cellBufferSize': 161120}, array([140396173240640, 140396172711552,            

In [ ]:
mesh1 = itk.mesh_from_dict(output_result[0][0])

#itkwidgets.view(geometries=[mesh1])


In [ ]:
itk.meshwrite(mesh1, 'mesh1.vtk')

In [ ]:
print(output_result[0][0])

{'meshType': {'dimension': 3, 'pointComponentType': 'float32', 'pointPixelComponentType': 'float64', 'pointPixelType': 'Scalar', 'pointPixelComponents': 1, 'cellComponentType': 'uint64', 'cellPixelComponentType': 'float64', 'cellPixelType': 'Scalar', 'cellPixelComponents': 1}, 'name': '', 'dimension': 3, 'numberOfPoints': 16587, 'points': array([ 55.538155,  91.491844,  64.33265 , ..., 104.95681 ,  93.99789 ,
        34.300396], dtype=float32), 'numberOfPointPixels': 16587, 'pointData': array([0.16583272, 0.        , 0.        , ..., 0.        , 0.10137601,
       0.        ]), 'numberOfCells': 32224, 'cells': array([140396173240640, 140396172711552,               0, ...,
                 16584,           16586,           16585], dtype=uint64), 'numberOfCellPixels': 32224, 'cellData': array([6.93649257e-310, 6.93649261e-310, 0.00000000e+000, ...,
       9.22310260e-002, 3.37920043e-002, 3.44735188e-002]), 'cellBufferSize': 161120}


In [ ]:
print(output_result[0][1])

[140396173240640 140396172711552               0 ...           16584
           16586           16585]


In [ ]:
print(output_result[0][2])

[140396173240640 140396172711552               0               0
               2]


In [ ]:
cells_array1 = output_result[0]#['cells']
print(cells_array1.shape[0])
cells_array1 = np.reshape(cells_array1, [32224, 5])
print(cells_array1.shape)

161120
(32224, 5)


In [ ]:
cells_array1[:3]

array([[2, 3, 0, 1, 2],
       [2, 3, 2, 3, 0],
       [2, 3, 4, 3, 2]], dtype=uint64)

In [ ]:
cells_array[:3]

array([[140396274906288, 140396248231168,               0,
                      0,               2],
       [              2,               3,               2,
                      3,               0],
       [              2,               3,               4,
                      3,               2]], dtype=uint64)

In [ ]:
# 140396274906288, 140396248231168

In [ ]:
cells_array = output_result[0]#['cells']
print(cells_array.shape[0])
cells_array = np.reshape(cells_array, [32224, 5])
print(cells_array.shape)

161120
(32224, 5)


In [ ]:
np.save('c1.npy', cells_array)

In [ ]:
for k in range(10):
  print(cells_array[k])

[140396274906288 140396248231168               0               0
               2]
[2 3 2 3 0]
[2 3 4 3 2]
[2 3 5 3 4]
[2 3 6 5 4]
[2 3 7 5 6]
[2 3 8 7 6]
[ 2  3  9 10 11]
[ 2  3 11 12  9]
[ 2  3 12 13 14]


In [ ]:
# To enable running the itkwidgets window on colab
from google.colab import output
output.enable_custom_widget_manager()

mesh1 = itk.mesh_from_dict(output_result[0])
itkwidgets.view(geometries=[mesh1])

RuntimeError: ignored

In [ ]:
# Read File from S3 from Dask workers inside the EC2 instances
# Now finish the pipeline

@delayed
def show_contents_s3_bucket_a(bucket_name, filename):
  import boto3
  import itk

  s3     = boto3.resource("s3")
  bucket = s3.Bucket(bucket_name)

  #all_objects = []
  #for obj in bucket.objects.all():
  #  all_objects.append(str(obj.key))
  #all_files = ",".join(all_objects)

  s3.Bucket(bucket_name).download_file(filename, filename)
  itk_image = itk.imread(filename)
  return itk_image.shape

bucket_name = 'oaisample1'
filename    = 'atlas_image.nii.gz'
output_str  = show_contents_s3_bucket_a(bucket_name, filename)

l = compute(output_str)
print(l)

In [ ]:
# For fetching files from S3

import boto3
s3 = boto3.resource("s3")

def show_contents_s3_bucket(bucket_name):
  import boto3

  s3     = boto3.resource("s3")
  bucket = s3.Bucket(bucket_name)

  all_objects = []
  for obj in bucket.objects.all():
    all_objects.append(str(obj.key))
  
  return ",".join(all_objects)

#print(f'filename : {obj.key} ')
#s3.Bucket(bucket_name).download_file(obj.key, './OAIData1/'+obj.key)
    
bucket_name = 'oaisample1'
show_contents_s3_bucket(bucket_name)

In [ ]:
# Code to create right transform

for i in range(phi_AB.GetNumberOfTransforms()):
  transform0 = phi_AB.GetNthTransform(i)
  transform1 = phi_AB1.GetNthTransform(i)

  o1 = transform1.GetFixedParameters()
  o1.SetSize(transform0['fixedParameters'].shape[0])
  for j, v in enumerate(transform0['fixedParameters']):
      o1.SetElement(j, v)
  transform1.SetFixedParameters(o1)

  o2 = transform1.GetParameters()
  o2.SetSize(transform0['parameters'].shape[0])
  for j, v in enumerate(transform0['parameters']):
      o2.SetElement(j, v)
  transform1.SetParameters(o2)

In [ ]:
#itk.dict_from_transform(phi_AB1)
phi_AB1.GetNthTransform(1)

<itk.itkTransformBasePython.itkTransformD33; proxy of <Swig Object of type 'itkTransformD33 *' at 0x7fa306063360> >

In [ ]:
transform1 = phi_AB1.GetNthTransform(1)
d1 = itk.dict_from_transform(transform1)
print(d1)
#print(transform1)

[{'name': '', 'numberOfTransforms': 1, 'parametersValueType': 'float64', 'inDimension': 3, 'outDimension': 3, 'transformName': 'DisplacementFieldTransform', 'transformType': 'D33', 'parameters': array([], dtype=float64), 'fixedParameters': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.]), 'numberOfParameters': 0, 'numberOfFixedParameters': 18}]


In [ ]:
phi_AB1_dis = phi_AB1.GetNthTransform(1)

In [ ]:
print(itk.dict_from_transform(phi_AB1_dis))

[{'name': '', 'numberOfTransforms': 1, 'parametersValueType': 'float64', 'inDimension': 3, 'outDimension': 3, 'transformName': 'DisplacementFieldTransform', 'transformType': 'D33', 'parameters': array([], dtype=float64), 'fixedParameters': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.]), 'numberOfParameters': 0, 'numberOfFixedParameters': 18}]


In [ ]:
itk.dict_from_transform(phi_AB1_dis)

[{'fixedParameters': array([192., 192.,  80.,   0.,   0.,   0.,   1.,   1.,   1.,   1.,   0.,
           0.,   0.,   1.,   0.,   0.,   0.,   1.]),
  'inDimension': 3,
  'name': '',
  'numberOfFixedParameters': 18,
  'numberOfParameters': 8847360,
  'numberOfTransforms': 1,
  'outDimension': 3,
  'parameters': array([-1.55766129, -1.21581364, -0.17686807, ...,  0.61738586,
          2.51548004,  9.465065  ]),
  'parametersValueType': 'float64',
  'transformName': 'DisplacementFieldTransform',
  'transformType': 'D33'}]

In [ ]:
# o2 = phi_AB1_dis.GetFixedParameters()
# print(o2.Size())

o2 = phi_AB1_dis.GetParameters()
o2.SetSize(d1[0]['parameters'].shape[0])
for j, v in enumerate(d1[0]['parameters']):
    o2.SetElement(j, v)
phi_AB1_dis.SetParameters(o2)

o1 = phi_AB1_dis.GetFixedParameters()
o1.SetSize(d1[0]['fixedParameters'].shape[0])
for j, v in enumerate(d1[0]['fixedParameters']):
    o1.SetElement(j, v)
phi_AB1_dis.SetFixedParameters(o1)

In [ ]:
itk.dict_from_transform(phi_AB1_dis)

[{'fixedParameters': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.]),
  'inDimension': 3,
  'name': '',
  'numberOfFixedParameters': 18,
  'numberOfParameters': 0,
  'numberOfTransforms': 1,
  'outDimension': 3,
  'parameters': array([], dtype=float64),
  'parametersValueType': 'float64',
  'transformName': 'DisplacementFieldTransform',
  'transformType': 'D33'}]

In [ ]:
itk.dict_from_transform(phi_AB1_dis)

[{'fixedParameters': array([192., 192.,  80.,   0.,   0.,   0.,   1.,   1.,   1.,   1.,   0.,
           0.,   0.,   1.,   0.,   0.,   0.,   1.]),
  'inDimension': 3,
  'name': '',
  'numberOfFixedParameters': 18,
  'numberOfParameters': 8847360,
  'numberOfTransforms': 1,
  'outDimension': 3,
  'parameters': array([-1.55766129, -1.21581364, -0.17686807, ...,  0.61738586,
          2.51548004,  9.465065  ]),
  'parametersValueType': 'float64',
  'transformName': 'DisplacementFieldTransform',
  'transformType': 'D33'}]

In [ ]:
phi_AB1_dis1 = itk.transform_from_dict(itk.dict_from_transform(phi_AB1_dis))
print(phi_AB1_dis1)

AttributeError: ignored

In [ ]:
import pickle
pickle.dump(d1, open("transform_object.pkl", "wb")) 

In [ ]:
print(phi_AB1.GetNthTransform(1)["fixedParameters"])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
print(phi_AB.GetNthTransform(1)["fixedParameters"])

[192. 192.  80.   0.   0.   0.   1.   1.   1.   1.   0.   0.   0.   1.
   0.   0.   0.   1.]


In [ ]:
temp1 = phi_AB1.GetNthTransform(1)
print(temp1["parameters"])
temp1["parameters"] = phi_AB.GetNthTransform(1)["parameters"]+1
print(temp1["parameters"])

[-1.55766129 -1.21581364 -0.17686807 ...  0.61738586  2.51548004
  9.465065  ]
[-0.55766129 -0.21581364  0.82313193 ...  1.61738586  3.51548004
 10.465065  ]


In [ ]:
itk.dict_from_transform(phi_AB1)

In [ ]:
itk.dict_from_transform(phi_AB)

In [ ]:
print(phi_AB.GetNthTransform(1)["parameters"])

[-1.55766129 -1.21581364 -0.17686807 ...  0.61738586  2.51548004
  9.465065  ]


In [ ]:
deformed_fc = deform_probmap_FC_delayed(phi_AB, image_A, image_B)

In [ ]:
phi_AB1 = itk.transform_from_dict(d1)

In [ ]:
phi_AB1["fixedParameters"]
print(phi_AB["fixedParameters"])

[]


In [ ]:
d1 = itk.dict_from_transform(phi_AB)

In [ ]:
print(phi_AB)

In [ ]:
d1

In [ ]:
d1 = itk.dict_from_transform(phi_AB)
phi_AB1 = itk.transform_from_dict(d1)

print(phi_AB1)

In [ ]:
phi_AB1_fixed_params = phi_AB1.GetNthTransform(0)#['fixedParameters']
print(phi_AB1_fixed_params["fixedParameters"])

phi_AB_fixed_params = phi_AB.GetNthTransform(0)#['fixedParameters']
print(phi_AB_fixed_params["fixedParameters"])

#print(phi_AB1_fixed_params.GetFixedParameters())

In [ ]:
import itk
import numpy as np
import pickle

Dimension = 3
PixelType = itk.D

# List of Transforms to test
transforms_to_test = [itk.DisplacementFieldTransform[PixelType, Dimension]]#[itk.AffineTransform[PixelType, Dimension], itk.DisplacementFieldTransform[PixelType, Dimension], itk.BSplineTransform[PixelType, Dimension, 3], itk.QuaternionRigidTransform[PixelType]]

keys_to_test1 = ["name", "parametersValueType", "transformName", "transformType", "inDimension", "outDimension", "numberOfParameters", "numberOfFixedParameters"]
keys_to_test2 = ["parameters", "fixedParameters"]

transform_object_list = []
for i, transform_type in enumerate(transforms_to_test):
    print(transform_type)

    transform = transform_type.New()
    transform.SetObjectName("transform"+str(i))
    print(transform.GetObjectName())

    fixedParameters = np.random.rand(3, 3, 2).flatten()
    print(transform.GetFixedParameters().Size())

    #print(fixedParameters.dtype)
    transform["fixedParameters"] = fixedParameters

    #print(transform["fixedParameters"])

    # Check the serialization
    serialize_deserialize = pickle.loads(pickle.dumps(transform))

    #print(serialize_deserialize)
    # # Test all the attributes
    for k in keys_to_test1:
        assert serialize_deserialize[k] == transform[k]

    # # Test all the parameters
    for k in keys_to_test2:
        print(k)
        print(serialize_deserialize[k])
        print(transform[k])
        assert np.array_equal(serialize_deserialize[k], transform[k])

    transform.SetFixedParameters(transform.GetFixedParameters())
    #transform["fixedParameters"] = fixedParameters#transform.GetFixedParameters()
    #print(transform)
    # transform_object_list.append(transform)

print('Individual Transforms Test Done')

ModuleNotFoundError: ignored

In [ ]:
transform.SetFixedParameters

In [ ]:
t1 = transform_object_list[1]
print(t1["fixedParameters"])

In [ ]:
transform_template = getattr(itk, 'DisplacementFieldTransform')
transform_template

In [ ]:
a = np.array([3, 3])
print(a.shape)
print(np.random.rand(*a))

In [ ]:
output_result = compute(result)
print(output_result)

In [ ]:
img_np = np.random.rand(5, 5)
print(img_np.shape)

img = itk.image_from_array(img_np)
print(img.shape)

In [ ]:
# Create a test mesh

m = itk.Mesh.F3.New()
p = np.random.rand(5, 3).astype('float32')
print(p.shape, p.dtype)
m.SetPoints(itk.vector_container_from_array(p.flatten()))

#md = itk.dict_from_mesh(m)
#print(md)

In [ ]:
@delayed
def perform_sum1(img_temp):
  import itk
  mesh = itk.mesh_from_dict(img_temp)
  return img_temp['dimension']

  #return np.sum(img_temp['points'])
  #return itk.di
  #import itkConfig
  #itkConfig.LazyLoading = False
  #import itk
  #return itk.image_from_array(img_temp)
  #a = np.array(img_temp)
  #return a#np.sum(img_temp)

In [ ]:
# Create Dask Computation Graph

result = []

#import_done = import_all()
image_preprocessed, atlas_image   = get_paths()

image_A, image_B, FC_prob, TC_prob  = read_images(image_preprocessed, atlas_image)
phi_AB            = only_register_images(image_A, image_B)

warped_image_FC = deform_probmap_FC(phi_AB, image_A, image_B, FC_prob)
warped_image_TC = deform_probmap_TC(phi_AB, image_A, image_B, TC_prob)

thickness_FC = get_thickness_FC(warped_image_FC)
thickness_TC = get_thickness_TC(warped_image_TC)

result.append(thickness_FC)
result.append(thickness_TC)

In [ ]:
numParam = deform_probmap_FC_paramA(phi_AB)


In [ ]:
m1 = itk.dict_from_mesh(m)
sumImg = perform_sum1(m1)

In [ ]:
#cluster.close()

In [ ]:
# Visualize Dask Computation Graph

visualize(sumImg)

In [ ]:
# Perform computation
#%%time
l = compute(sumImg)

In [ ]:
print(l)

In [ ]:
import pickle
import cloudpickle
a = itk.image_from_array(img_np)
b = cloudpickle.loads(pickle.dumps(a))
print(b)

In [ ]:
!pip install cloudpickle

![DaskProgress](https://user-images.githubusercontent.com/1044135/163692457-40a41395-3d83-4928-9445-528cd025a7b9.png)


In [ ]:
# Write the result meshes

distance_inner_FC = l[0]
distance_inner_TC = l[1]

itk.meshwrite(distance_inner_FC, 'itk_distance_inner_FC.vtk')
itk.meshwrite(distance_inner_TC, 'itk_distance_inner_TC.vtk')

In [ ]:
# Visualize the meshes with thickness as cell data

# To enable running the itkwidgets window on colab
from google.colab import output
output.enable_custom_widget_manager()

a1 = vtk.vtkPolyDataReader()
a1.SetFileName('itk_distance_inner_FC.vtk')
a1.Update()
distance_inner_FC = a1.GetOutput()

itkwidgets.view(geometries=[distance_inner_FC])